# Sumarizador abstrato de texto
Nesse notebook serão feitos o passo-a-passo para a implementação de um sumarizador de texto através se uma rede neural RNN LSTM consituida de um encoder e um decoder com um sistema de atenção implementado nas hidden layers.


O primeiro passo será importar todas as bibliotecas necessárias e fazer a analise e limpesa do dataset

In [0]:
# Importando as bibliotecas de manipulação e exploração de dados
import numpy as np
import os
import pandas as pd
import spacy
import re
import nltk
from nltk.corpus import stopwords
pd.set_option("display.max_colwidth", 500)

In [0]:
4/swHX0Ik-0tQEC1PaBxADzYZiXcII-KHcUs6kt__jdEL6SqEqCd6J7WY

In [2]:
from google.colab import files, drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
src = list(files.upload().values())[0]

Saving attention.py to attention.py


In [4]:
open('mylib.py','wb').write(src)

5230

In [5]:
#importando a biblioteca da layer de atenção que não se encontra no keras
from attention import AttentionLayer

In [6]:
#importando as bibliotecas de DeepLearning
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
# Abrindo o dataset em um arquivo pandas
df = pd.read_csv('/content/drive/My Drive/TCC/Dados/tcu_acordaos2.csv')

In [0]:
# Eliminando duplicatcas e arquivos em branco
df = df[['sumario','acordao']]
df.drop_duplicates(subset=['acordao'],inplace=True)
df.dropna(axis=0,inplace=True)

## Nós vamos performar os seguintes pré-processamentos nos nossos dados:

1.Converter tudo para caixa baixa

4.Remover (‘s)

5.Remover texto de dentro de parenteses ()

6.Eliminar pontuação e caracteres especiais

7.Remover stopwords

8.Remover palavras curtas

Definiremos a nossa função como:

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def texto_limpo(text, sumario=False):
  if not sumario:
    # Convertendo para lower case
    text_limpo = text.lower()
    # Eliminando caracteres especiais
    text_limpo = re.sub(r'[_"\-;%()¿|+º&=*%!.,:#$@\[\]/]', ' ', text_limpo)
    # Eliminando espaços duplos
    text_limpo = re.sub(r'\s\s', ' ', text_limpo)
    # Eliminando números
    text_limpo = re.sub(r'[0-9]', '', text_limpo)
    # Eliminando palavras pequenas
    tokens = text_limpo.split()
    palavras_grandes=[]
    for i in tokens:
      if len(i)>=3:
        palavras_grandes.append(i)  

    text_limpo = " ".join(palavras_grandes)

    text_limpo = text_limpo.split()
    stops = set(stopwords.words("portuguese"))
    text_limpo = [w for w in text_limpo if not w in stops]
    text_limpo = " ".join(text_limpo)

  if sumario:
    # Convertendo para lower case
    text_limpo = text.lower()
    # Eliminando caracteres especiais
    text_limpo = re.sub(r'[_"\-;%()¿|+º&=*%!,:#$@\[\]/]', ' ', text_limpo)
    # Tornando ponto  em tokens
    text_limpo = re.sub(r'\.', ' .', text_limpo)
    # Eliminando espaços duplos
    text_limpo = re.sub(r'\s\s+', ' ', text_limpo)
    # Eliminando números
    text_limpo = re.sub(r'[0-9]', '', text_limpo)
    # Eliminando palavras pequenas
    tokens = text_limpo.split()
    palavras_grandes=[]
    for i in tokens:
      if len(i)>1:
        palavras_grandes.append(i)

  return text_limpo

In [0]:
#limpando os acordaos
acordao_limpo = []
for t in df['acordao']:
   acordao_limpo.append(texto_limpo(t,sumario = False))

In [0]:
#Limapando os sumarios
sumario_limpo = []
for t in df['sumario']:
    sumario_limpo.append(texto_limpo(t, sumario = True))

In [0]:
df['acordao']=acordao_limpo
df['sumario']=sumario_limpo

In [20]:
#Definindo o tamanho maximo dos sumarios
cnt=0
for i in df.sumario:
    if(len(i.split())<=100):
        cnt=cnt+1
print(cnt/len(df.sumario))

0.992668763945286


In [21]:
# Definindo o tamanho máximo dos acordãos
cnt=0
for i in df.acordao:
    if(len(i.split())<=300):
        cnt=cnt+1
print(cnt/len(df.acordao))

1.0


In [0]:
#Definindo os valores finais
max_len_text=300
max_len_summary=100

In [0]:
#Eliminando os textos muito grandes
cleaned_acordao =np.array(df.acordao)
cleaned_sumario=np.array(df.sumario)

short_acordao=[]
short_sumario=[]

for i in range(len(cleaned_acordao)):
    if(len(cleaned_sumario[i].split())<=max_len_summary and len(cleaned_acordao[i].split())<=max_len_text):
        short_acordao.append(cleaned_acordao[i])
        short_sumario.append(cleaned_sumario[i])
        
df=pd.DataFrame({'acordao':short_acordao,'sumario':short_sumario})

In [0]:
# Incluindo os tokens de inicio e final de sentença
df['sumario'] = df['sumario'].apply(lambda x : 'xxbos '+ x + ' xxeos')

In [0]:
# Dividindo em test e validação
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['acordao']),np.array(df['sumario']),test_size=0.2,random_state=3,shuffle=True) 

In [0]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#Prepare o tokenizador dos acordãos
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

#Converter texto em sequencia de números
x_tr    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#Fazendo padding para o length máximo
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post') 
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [0]:
#Prepare o tokenizador dos sumarios
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

#Converter texto em sequencia de números
y_tr    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val   =   y_tokenizer.texts_to_sequences(y_val) 

#Fazendo padding para o length máximo
y_tr    =   pad_sequences(y_tr, maxlen=max_len_summary, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_summary, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

In [0]:
# Dicionarios trazendo de volta index para palavra
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index

In [0]:
from keras import backend as K 
K.clear_session() 
latent_dim = 256

# Encoder 
encoder_inputs = Input(shape=(max_len_text,)) 
enc_emb = Embedding(x_voc_size, latent_dim,trainable=True)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

#LSTM 3 
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

#------------------------------------------------------------------------------#

# Inicializando o decoder 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(y_voc_size, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM usando encoder_states como estados iniciais
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

#Layer de Atenção
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

#Concatenar a saida da layer de atenção com a saida do decoder
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Layer de probabilidade distribuida no vocabulario
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

# Define o modelo
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 

# Carregando os pesos previamente treinados
model.load_weights('/content/drive/My Drive/TCC/modelos/modelo1:011-val_acc:0.810.hdf5')

In [0]:
# Codifica a sequência de entrada para obter o vetor de features
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Configuração do Decoder
# Os tensors vão guardar os estados dos periodos de tempos passados
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

# Embbeding do decodificador
dec_emb2= dec_emb_layer(decoder_inputs) 
# Para prever a próxima palavra defina o estado inicial como o estado no periodo anterior
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#Inferencia por Atenção
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# Camada densa que vai gerar a distribuição de probabilidade em cima do vocabulário
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Modelo final do decoder
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [0]:
from random import choice

def select_nucleus(outp, p=0.5):
  # Seleciona aleatoriamente entre as palavras que tem probalidade acumulada
  # de pelo menos 50% ou valor definido
    probs = outp.reshape(-1)
    idxs = np.argsort(-probs)
    res,cumsum = [],0.
    for idx in idxs:
        res.append(idx)
        cumsum += probs[idx]
        if cumsum > p: return choice(res)

In [0]:
def decode_sequence(input_seq):
    # Condifica o input como um vetor de estado
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Gera uma sequência alvo de tamanho 1
    target_seq = np.zeros((1,1))
    
    # Define a primeira palavra como sendo xxbos
    target_seq[0, 0] = target_word_index['xxbos']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Prevê a próxima palavra
        sampled_token_index = select_nucleus(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='xxeos'):
            decoded_sentence += ' '+sampled_token

        # Condição de saida: Prevê o token xxoes ou atinge o tamanho máximo estabelecido para o sumário.
        if (sampled_token == 'xxeos'  or len(decoded_sentence.split()) >= (max_len_summary-1)):
            stop_condition = True

        # Atualiza a palavra que sera usada para prever a próxima.
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Atualiza os estados internos
        e_h, e_c = h, c

    return decoded_sentence

In [0]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['xxbos']) and i!=target_word_index['xxeos']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [0]:
for i in range(5):
    print("Acordão:",seq2text(x_val[i]))
    print("Sumário Original:",seq2summary(y_val[i]))
    print("Sumário gerado:",decode_sequence(x_val[i].reshape(1,max_len_text)))
    print("\n")

# Melhores e Piores Exemplos

In [0]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import random

scores = []

mylist = list(range(len(y_val)))

bleu_list = random.sample(mylist,500)

for i in bleu_list:

  sumarios_produzidos = decode_sequence(x_val[i].reshape(1,max_len_text)).split()
  sumarios_originais = [seq2summary(y_val[i]).split()]

  score_bleu = sentence_bleu(sumarios_originais, sumarios_produzidos)

  scores.append(score_bleu)

scores_ordenados = np.argsort(-np.array(scores))

In [39]:
for i in scores_ordenados[:10]:
  print(scores[i])

1.0
1.0
1.0
0.9365905284849905
0.8827916928185874
0.8696398662122882
0.8667865171073738
0.8667865171073738
0.7639843504088955
0.749634235443537


In [43]:
for i in scores_ordenados[:15]:
    print("Acórdão:",seq2text(x_val[bleu_list[i]]))
    print("Sumário Original:",seq2summary(y_val[bleu_list[i]]))
    print("Sumário gerado:",decode_sequence(x_val[bleu_list[i]].reshape(1,max_len_text)))
    print("\n")

Acordão: vistos relatados discutidos autos tomada contas especial responsabilidade floriano rodrigues rocha decorrência inadimplemento omissão prestação contas acordo firmado lba considerando processo devidamente organizado apurou débito contra responsável valor ncz considerando devidamente citado responsável apresentou alegações defesa recolheu valor débito imputado considerando revel termos art lei acordam ministros tribunal contas união reunidos sessão câmara fundamento arts inciso inciso iii alínea lei arts inciso iii mesma lei julgar presentes contas irregulares condenar floriano rodrigues rocha pagamento quantia ncz seiscentos noventa cruzados novos trinta sete centavos acrescida encargos legais calculados partir data efetivo recolhimento convertida padrão monetário vigente fixação prazo quinze dias contar notificação comprovar perante tribunal art inciso iii alínea regimento interno recolhimento dívida cofres autorizar desde logo termos art inciso lei cobrança judicial dívida ca

In [41]:
for i in scores_ordenados[-10:]:
  print(scores[i])

0.01112197173469177
0.010962589591484207
0.0065154316435699884
0.005954630418082279
0.004906798012049312
0.004224577864166247
0.003598554902990897
0.002824569767362339
0.0023136228914154575
0.0008311149983222482


In [44]:
for i in scores_ordenados[-10:]:
    print("Acórdão:",seq2text(x_val[bleu_list[i]]))
    print("Sumário Original:",seq2summary(y_val[bleu_list[i]]))
    print("Sumário gerado:",decode_sequence(x_val[bleu_list[i]].reshape(1,max_len_text)))
    print("\n")

Acórdão: vistos relatados discutidos autos tomada contas especial instaurada coordenação regional fundação nacional saúde estado pará decorrência irregularidades execução convênio firmado prefeitura municipal bonito acordam ministros tribunal contas união reunidos sessão primeira câmara fundamento arts inciso inciso iii alíneas inciso iii lei arts incisos inciso iii regimento interno ante razões expostas relator rejeitar alegações defesa apresentadas jamil assad neto julgar irregulares contas condenando pagamento quantias abaixo discriminadas atualizadas monetariamente acrescidas juros mora calculados partir respectivas datas ocorrência efetiva quitação débito fixando prazo quinze dias desde ciência comprovem perante tribunal recolhimento referida quantia cofres cofres fundo nacional saúde valor original data ocorrência aplicar jamil assad neto multa prevista art lei art regimento interno tcu valor duzentos mil reais fixando prazo quinze dias contar notificação comprove perante tribuna

## Metrica Bleu

In [36]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import random

scores = []

mylist = list(range(len(y_val)))
for a in range(10):
  bleu_list = random.sample(mylist,100)

  for i in bleu_list:

    sumarios_produzidos = decode_sequence(x_val[i].reshape(1,max_len_text)).split()
    sumarios_originais = [seq2summary(y_val[i]).split()]

    score_bleu = sentence_bleu(sumarios_originais, sumarios_produzidos,weights=(1, 0, 0, 0))

    scores.append(score_bleu)

  Bleu = np.array(scores).mean()
  print('Bleu:', str(round(Bleu*100,3))+'%')

Bleu: 30.346%
Bleu: 33.498%
Bleu: 33.668%
Bleu: 32.381%
Bleu: 31.914%
Bleu: 32.182%
Bleu: 31.732%
Bleu: 31.447%
Bleu: 31.192%
Bleu: 31.063%


## Metrica Rouge

In [34]:
!pip install rouge

In [37]:
from rouge import Rouge
import random

mylist = list(range(len(y_val)))
for a in range(10):
  rouge_list = random.sample(mylist,100)

  rouge = Rouge()
  sumarios_originais = [seq2summary(y_val[i]) for i in rouge_list]
  sumarios_produzidos = [decode_sequence(x_val[i].reshape(1,max_len_text)) for i in rouge_list]
                                        
  score_rouge = rouge.get_scores(sumarios_originais, sumarios_produzidos, avg=True)
  print(score_rouge)

{'rouge-1': {'f': 0.40742673118463174, 'p': 0.40768158529724063, 'r': 0.4656702451382449}, 'rouge-2': {'f': 0.22017003476982708, 'p': 0.22151533957267897, 'r': 0.2527791748817244}, 'rouge-l': {'f': 0.32432524805766766, 'p': 0.351901605154966, 'r': 0.4034360555193146}}
{'rouge-1': {'f': 0.4068943770092684, 'p': 0.4136535964588291, 'r': 0.4471518801585745}, 'rouge-2': {'f': 0.22619045725759762, 'p': 0.2289246987901166, 'r': 0.24776414851622486}, 'rouge-l': {'f': 0.3335895044638296, 'p': 0.36220782816300323, 'r': 0.39290029263579174}}
{'rouge-1': {'f': 0.4083556611538618, 'p': 0.38849312834947336, 'r': 0.4775052266373339}, 'rouge-2': {'f': 0.1990651863064471, 'p': 0.19170177834661406, 'r': 0.23256052478352526}, 'rouge-l': {'f': 0.3235020583765062, 'p': 0.3336205731502988, 'r': 0.40679434191696406}}
{'rouge-1': {'f': 0.42794774774771094, 'p': 0.3993624446774559, 'r': 0.5055347959205775}, 'rouge-2': {'f': 0.2508777628823163, 'p': 0.2374774710108332, 'r': 0.29308606132344084}, 'rouge-l': {'f